# 問題

## (1) 物体検出の分野にはどういった手法が存在したか。<br>

* Selective Search
  + ピクセルごとにグループ化する
  + 下記の特徴がある（原著参照）
    1. 物体毎の境界を明確にする
    2. 色が近いもの同士のセグメントも出来る
    3. 既存の物体認識より素早い計算が出来る
  
* Sliding Windows
  + Edge Boxesなど
  
<br>
<br>

### 【Selective Search for Object Recognition】
https://staff.fnwi.uva.nl/th.gevers/pub/GeversIJCV2013.pdf
#### 3 Selective Search
>"Capture All Scales. Objects can occur at any scale within the image. Furthermore, some objects have less clear boundaries then other objects.<br> 
Therefore, in selective search all object scales have to be taken into account, as illustrated in Fig. 2. This is most naturally achieved by using an hierarchical algorithm.<br>
<br>
Diversification. There is no single optimal strategy to group regions together. As observed earlier in Fig. 1, regions may form an object because of only colour, only texture, or because parts are enclosed. <br>
Furthermore, lighting conditions such as shading and the colour of the light may influence how regions form an object. <br>
Therefore instead of a single strategy which works well in most cases, we want to have a diverse set of strategies to deal with all cases.<br>
<br>
Fast to Compute. The goal of selective search is to yield a set of possible object locations for use in a practical object recognition framework. <br>
The creation of this set should not become a computational bottleneck, hence our algorithm should be reasonably fast."


#### 2 RELATED WORK
##### Object Proposals. 
> There is a large literature on object proposal methods. <br>
Comprehensive surveys and comparisons of object proposal methods can be found in [19], [20], [21]. <br>
Widely used object proposal methods include those based on grouping super-pixels (e.g., Selective Search [4], CPMC [22], MCG [23]) and those based on sliding windows (e.g., objectness in windows [24], EdgeBoxes [6]). <br>
Object proposal methods were adopted as external modules independent of the detectors (e.g., Selective Search [4] object detectors, R- CNN [5], and Fast R-CNN [2]).<br>

## (2) Fasterとあるが、どういった仕組みで高速化したのか。<br>

* RPN (Region Proposal Networks)
* Fast R-CNN

* 上2つを組み合わせたAttention機構
  + RPN は Fast R-CNN のどこに注意を向けるべきか示してくれる
  + RPN はスケールの異なるアンカーボックスを使うことで処理を簡素化する
<br>
<br>

#### 3 FASTER R-CNN
> "Using the recently popular terminology of neural networks with ‘attention’ [31] mechanisms, the RPN module tells the Fast R-CNN module where to look."

##### Multi-Scale Anchors as Regression References
> "The design of multi-scale anchors is a key component for sharing features without extra cost for addressing scales."


## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。<br>

* one-stage(e.g. OverFeat): クラス特定の検出パイプライン
  + 領域毎の特徴は、１種類のアスペクト比・スケールのピラミッドから生成された１枚の sliding Window から成り立っている

* two-stage(e.g. RPN): クラス無視の提案、クラス特定の検出
  +  領域毎の特徴は $ 3*3$ サイズの sliding Windows から成っていて 、異なるアスペクト比・スケールを持つアンカーボックスと紐付いている提案領域を予測する

* one-stage は提案領域の数が多い為、処理が遅い
<br>
<br>

#### 【4 EXPERIMENTS】
##### 4.1 Experiments on PASCAL VOC
##### One-Stage Detection vs. Two-Stage Proposal + Detection.

>"In OverFeat, the region-wise features come from a sliding window of one aspect ratio over a scale pyramid. <br>
These features are used to simultaneously determine the location and category of objects. <br>
In RPN, the features are from square (3×3) sliding windows and predict proposals relative to anchors with different scales and aspect ratios.<br> 
Though both methods use sliding windows, the region proposal task is only the first stage of Faster R-CNN—the downstream Fast R-CNN detector attends to the proposals to refine them.<br> 
In the second stage of our cascade, the region-wise features are adaptively pooled [1], [2] from proposal boxes that more faithfully cover the features of the regions. <br>
We believe these features lead to more accurate detections."<br>
<br>
>"We also note that the one-stage system is slower as it has considerably more proposals to process."<br>

## (4) RPNとは何か。<br>

* Fast R-CNN が行う畳み込み演算を分担して行うことで計算時間を削減する
  + $n * n$ の疎な sliding windows から得られた特徴を入力の特徴マップに落とし込み、２つの $1 * 1$ 畳み込み層に渡す
    - a box-regression layer (reg)
      + バウンディングボックスの位置ズレを回帰で予測する
    - a box-classification layer (cls)
      + 物体かどうかの２値分類問題を解く(IoU)
  
<br>
<br>

##### 3.1 Region Proposal Networks
> "A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score."<br>
> "Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers. " <br>


## (5) RoIプーリングとは何か。<br>

* 特徴マップから、長さが補正された特徴ベクトルを抽出
  + 各特徴ベクトルはFC層に渡され、２つの出力層に分岐する
    - Kクラス分類を行う層
    - Kクラス内の１つのバウンディングボックスの位置情報を出力する層
    
<br>
<br>

#### 【Fast R-CNN】
https://arxiv.org/pdf/1504.08083.pdf
##### 2. Fast R-CNN architecture and training

>"for each object proposal a region of interest (RoI) pooling layer extracts a fixed-length feature vector from the feature map. <br>
Each feature vector is fed into a sequence of fully connected (fc) layers that finally branch into two sibling output layers:<br>
    one that produces softmax probability estimates over K object classes plus a catch-all “background” class <br>
    and another layer that outputs four real-valued numbers for each of the K object classes. <br>
    Each set of 4 values encodes refined bounding-box positions for one of the K classes."<br>

##### 2.1. The RoI(Region of Interest) pooling layer

>"The RoI pooling layer uses max pooling to convert the features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7), <br>
where H and W are layer hyper-parameters that are independent of any particular RoI. In this paper, <br>
an RoI is a rectangular window into a conv feature map. <br>
Each RoI is defined by a four-tuple (r, c, h, w) that specifies its top-left corner (r, c) and its height and width (h, w)."<br>


## (6) Anchorのサイズはどうするのが適切か。<br> 

* 各 sliding position の Anchor のサイズ：$k=9$ (3 scales × 3 aspect ratios)
  + $W * H$ の特徴マップのトータルのAnchorは $W * H * k$
  
<br> 
<br>

#### 3 FASTER R-CNN
##### 3.1.1 Anchors
>    "An anchor is centered at the sliding window in question, and is associated with a scale and aspect ratio (Figure 3, left). <br>
By default we use 3 scales and 3 aspect ratios, yielding k = 9 anchors at each sliding position. <br>
For a convolutional feature map of a size W × H (typically ∼2,400), there are W H k anchors in total." <br>

##### Translation-Invariant Anchors
>    "MultiBox has a (4 + 1) × 800-dimensional fully-connected output layer, <br>
     whereas our method has a (4 + 2) × 9-dimensional convolutional output layer in the case of k = 9 anchors."


## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。<br>

* MS COCO
  + 同条件のFast R-CNN と Faster R-CNN とを比べる
  + COCO val, test-dev 両方で、mAPが3ポイント程Fasterの方が高い
  
<br>
<br>

#### 4.2 Experiments on MS COCO
##### Table 11

>"Next we evaluate our Faster R-CNN system. <br>
Using the COCO training set to train, Faster R-CNN has 42.1% mAP@0.5 and 21.5% mAP@[.5, .95] on the COCO test-dev set. <br>
This is 2.8% higher for mAP@0.5 and 2.2% higher for mAP@[.5, .95] than the Fast R- CNN counterpart under the same protocol (Table 11). <br>
This indicates that RPN performs excellent for improving the localization accuracy at higher IoU thresh- olds. <br>
Using the COCO trainval set to train, Faster R-CNN has 42.7% mAP@0.5 and 21.9% mAP@[.5, .95] on the COCO test-dev set. <br>
Figure 6 shows some results on the MS COCO test-dev set."<br>




## (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

### 【You Only Look Once: Unified, Real-Time Object Detection】
https://arxiv.org/abs/1506.02640
#### 4. Experiments
##### 4.1. Comparison to Other Real-Time Systems<br>

> "The recent Faster R-CNN replaces selective search with a neural network to propose bounding boxes, similar to Szegedy et al. [8] <br>
In our tests, their most accurate model achieves 7 fps while a smaller, less accurate one runs at 18 fps. <br>
The VGG-16 version of Faster R-CNN is 10 mAP higher but is also 6 times slower than YOLO.<br>
The Zeiler-Fergus Faster R-CNN is only 2.5 times slower than YOLO but is also less accurate."<br>

### 【SSD: Single Shot MultiBox Detector】
https://arxiv.org/abs/1512.02325
#### 2 The Single Shot Detector (SSD)
##### 2.1 Model
##### Default boxes and aspect ratios
> "Our default boxes are similar to the anchor boxes used in Faster R-CNN [2], however we apply them to several feature maps of different resolutions. <br>
Allowing different default box shapes in several feature maps let us efficiently discretize the space of possible output box shapes."
